# PointNet Verification with α,β-CROWN on Google Colab

This notebook verifies PointNet robustness using α,β-CROWN.
Google Colab provides GPUs with 15GB VRAM (T4), enough for verification with large epsilon values.

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install dependencies
!pip install torch numpy onnx onnxruntime pyyaml packaging
!git clone https://github.com/Verified-Intelligence/alpha-beta-CROWN.git

In [ ]:
# Setup files - assumes you uploaded:
# - pointnet.pth
# - test_groups.npy  
# - test_labels.npy
# to /content/

import os
import shutil

# Create folders
os.makedirs('models', exist_ok=True)
os.makedirs('data/pointnet', exist_ok=True)

# Copy files to correct locations
shutil.copy('/content/pointnet.pth', 'models/pointnet.pth')
shutil.copy('/content/test_groups.npy', 'data/pointnet/test_groups.npy')
shutil.copy('/content/test_labels.npy', 'data/pointnet/test_labels.npy')

print("Files ready!")
!ls -la models/
!ls -la data/pointnet/

In [ ]:
# PointNet Model Definition
import torch
import torch.nn as nn
import torch.nn.functional as F

class PointNetForVerification(nn.Module):
    """
    PointNet for verification with T-Net (Spatial Transformer).
    Based on: Qi et al., "PointNet: Deep Learning on Point Sets" (CVPR 2017)
    """

    def __init__(self, num_points=64, num_classes=2, use_tnet=True, pooling="max"):
        super().__init__()
        self.num_points = num_points
        self.num_classes = num_classes
        self.use_tnet = use_tnet
        self.pooling = pooling

        if use_tnet:
            # Input T-Net (3x3 transformation matrix)
            self.tnet_conv1 = nn.Conv1d(3, 64, 1)
            self.tnet_conv2 = nn.Conv1d(64, 128, 1)
            self.tnet_conv3 = nn.Conv1d(128, 256, 1)
            self.tnet_fc1 = nn.Linear(256, 128)
            self.tnet_fc2 = nn.Linear(128, 64)
            self.tnet_fc3 = nn.Linear(64, 9)
            self.tnet_fc3.weight.data.zero_()
            self.tnet_fc3.bias.data.copy_(torch.eye(3).view(-1))

        # Point-wise MLP
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 256, 1)

        # Classifier
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        batch_size = x.shape[0]
        if x.dim() == 2:
            x = x.view(batch_size, self.num_points, 3)

        if self.use_tnet:
            x_t = x.transpose(1, 2)
            t = F.relu(self.tnet_conv1(x_t))
            t = F.relu(self.tnet_conv2(t))
            t = F.relu(self.tnet_conv3(t))
            if self.pooling == "mean":
                t = torch.mean(t, dim=2)
            else:
                t = torch.max(t, dim=2)[0]
            t = F.relu(self.tnet_fc1(t))
            t = F.relu(self.tnet_fc2(t))
            t = self.tnet_fc3(t)
            t = t.view(batch_size, 3, 3)
            x = torch.bmm(x, t)

        x = x.transpose(1, 2)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        if self.pooling == "mean":
            x = torch.mean(x, dim=2)
        else:
            x = torch.max(x, dim=2)[0]

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

print("PointNet model defined!")

In [ ]:
# Setup alpha-beta-CROWN
import sys
sys.path.insert(0, '/content/alpha-beta-CROWN/complete_verifier')

import numpy as np
from api import ABCrownSolver, ConfigBuilder, VerificationSpec, input_vars, output_vars

print("α,β-CROWN imported successfully!")

In [ ]:
# Load model and data
model = PointNetForVerification(num_points=64, num_classes=2, use_tnet=True, pooling="max")
checkpoint = torch.load('models/pointnet.pth', map_location='cpu', weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

X_test = np.load('data/pointnet/test_groups.npy')
y_test = np.load('data/pointnet/test_labels.npy')

print(f"Model loaded successfully!")
print(f"Test samples: {len(X_test)}")
print(f"Sample shape: {X_test[0].shape}")

In [ ]:
def verify_sample(model, sample, label, epsilon, timeout=120):
    """Verify a single sample using alpha-beta-CROWN."""
    if sample.shape == (3, 64):
        sample_flat = sample.T.flatten()
    else:
        sample_flat = sample.flatten()

    x = input_vars(192)
    y = output_vars(2)

    input_constraint = (x >= sample_flat - epsilon) & (x <= sample_flat + epsilon)
    output_constraint = y[label] > y[1 - label]

    spec = VerificationSpec.build_spec(
        input_vars=x, output_vars=y,
        input_constraint=input_constraint,
        output_constraint=output_constraint,
    )

    cfg = ConfigBuilder.from_defaults().set(
        bab__timeout=timeout,
        general__enable_incomplete_verification=True,
        general__complete_verifier="bab",
        general__conv_mode="matrix",
        attack__pgd_order="before",
        attack__pgd_steps=100,
        attack__pgd_restarts=30,
    )

    solver = ABCrownSolver(spec, model, config=cfg)
    result = solver.solve()

    verified = result.status in ["safe", "safe-incomplete", "verified", "unsat"]
    return {"status": result.status, "verified": verified}

print("Verification function ready!")

In [ ]:
# Run verification for multiple epsilon values
# 0.04 ~ 6cm, 0.05 ~ 7.5cm (comparable to NSGA-III perturbations)
epsilons = [0.005, 0.01, 0.02, 0.04, 0.05]
n_samples = 10

results = {}

for eps in epsilons:
    cm = eps * 150
    print(f"\n{'='*60}")
    print(f"Epsilon: {eps} (~{cm:.1f} cm)")
    print("="*60)

    verified_count = 0
    unsafe_count = 0

    for i in range(n_samples):
        sample = X_test[i]
        label = int(y_test[i])

        try:
            result = verify_sample(model, sample, label, eps)
            status = "VERIFIED" if result["verified"] else f"UNSAFE ({result['status']})"
            if result["verified"]:
                verified_count += 1
            else:
                unsafe_count += 1
        except Exception as e:
            status = f"ERROR: {str(e)[:50]}"
            unsafe_count += 1

        print(f"  Sample {i} (label={label}): {status}")

    results[eps] = {
        "cm": cm,
        "verified": verified_count,
        "unsafe": unsafe_count,
        "verified_pct": 100 * verified_count / n_samples
    }

    print(f"\nSummary: Verified={verified_count}/{n_samples} ({100*verified_count/n_samples:.1f}%)")

In [ ]:
# Final Summary
print("\n" + "="*60)
print("FINAL RESULTS - PointNet MAX-POOLING with T-Net")
print("="*60)
print(f"{'Epsilon':>8} | {'~cm':>6} | {'Verified':>10} | {'Unsafe':>10}")
print("-"*45)
for eps, r in results.items():
    print(f"{eps:>8.3f} | {r['cm']:>5.1f}  | {r['verified_pct']:>8.1f}%  | {100-r['verified_pct']:>8.1f}%")

print("\n" + "="*60)
print("Comparison with NSGA-III (SLAM attack):")
print("  NSGA-III perturbations: 0.98 - 5.02 cm")
print("  SLAM baseline error: 23 cm")
print("="*60)

In [ ]:
# Save results
import json
from google.colab import files

with open('verification_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved!")
print(json.dumps(results, indent=2))

# Download results
files.download('verification_results.json')